In [1]:
import os
from utils import *
import numpy as np
import cv2
import pandas as pd
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
import pytesseract
from tqdm import tqdm

In [2]:
images_names = []
for roots, dirs, files in os.walk('./data/split_img'):
    images_names.extend(dirs)

In [3]:
df = pd.read_csv('./data/csv/combined-training.csv')

In [4]:
def rotate(image, pt1, pt2):
    rows = image.shape[0]
    cols = image.shape[1]
    
    x0, y0 = pt1[0], pt1[1]
    x1, y1 = pt2[0], pt2[1]
    if x1 == x0:
        x1 = x0+1
        
    incline = abs((y1-y0)/(x1-x0))
    
    if incline <1:
        if x1 > x0:
            return image
        else:
            # right 180
            rotated_image = cv2.flip(image, -1)
    else:
        if y1>y0:
            # left 90
            M = cv2.getRotationMatrix2D((cols/2,cols/2),90,1)
        else:
            # right 90
            M = cv2.getRotationMatrix2D((rows/2,rows/2),270,1)
    
        rotated_image = cv2.warpAffine(image,M,(rows, cols))
    
    return rotated_image

In [5]:
X = []
y = []
sub_imgs = []
pt12s = []
i = 0
for image_n in tqdm(images_names):
    
    img = cv2.imread(os.path.join('./data/img',image_n+'.jpg'))
    
    for j in range(7):
        xys = df[df['Image-name'] == (image_n+'.jpg')].iloc[0,j*10+1:j*10+10]
        xys = list(xys)
        
        if xys[0] == 1:
            y.append(j+1)
            
            sub_img = four_point_transform(img, np.reshape(a=xys[1:], newshape=(4,2)))   
            cv2.imwrite('./data/train_img/{}.jpg'.format(str(i)), sub_img)
            pt1, pt2 = [xys[1], xys[2]], [xys[3], xys[4]]
            pt12s.append([pt1, pt2])
            sub_img = rotate(sub_img, pt1, pt2)
            cv2.imwrite('./data/train_img_/{}.jpg'.format(str(i)), sub_img)
            i += 1
            chars = pytesseract.image_to_string(sub_img)
#             print(chars)
            X.append(chars)
#     pytesseract.image_to_string(cv2.flip(sub_img, -1))
X = [X[i].replace('\n', ' ') for i in range(len(X))]

100%|██████████| 696/696 [06:14<00:00,  1.86it/s]


In [30]:
img_ = cv2.imread('./data/train_img_/3.jpg')

In [31]:
from PIL import Image                               
im = Image.open('./data/train_img_/3.jpg')

In [32]:
im = im.resize((img_.shape[1]*5, img_.shape[0]*5))

In [33]:
im = np.array(im)

In [10]:
np.save('X.npy', X)
np.save('y.npy', y)